In [ ]:
import warnings
warnings.filterwarnings("ignore")
from plotnine import ggplot, aes, geom_point, geom_map
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error, r2_score
style.use('fivethirtyeight')
%matplotlib inline
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import pandas as pd
import dash_bootstrap_components as dbc
import plotly.express as px
from plotly import graph_objs as go


app = dash.Dash(__name__,external_stylesheets=[dbc.themes.BOOTSTRAP])
server = app.server

colors = {
    'background': '#111111',
    'text': '#7FDBFF'
}


df1 = pd.read_csv("airline_codes.csv")
df2 = pd.read_csv("airport_codes.csv")

Airlines = df1['Airline Name'].unique()
Airport = df2['Airport Name'].unique()

# the style arguments for the sidebar. We use position:fixed and a fixed width
SIDEBAR_STYLE = {
    "position": "fixed",
    "top": 0,
    "left": 0,
    "bottom": 0,
    "width": "20rem",
    "padding": "2rem 1rem",
    "background-color": "#f8f9fa",
}

# the styles for the main content position it to the right of the sidebar and
# add some padding.
CONTENT_STYLE = {
    "margin-left": "22rem",
    "margin-right": "2rem",
    "padding": "2rem 1rem",
}



sidebar = html.Div(
    [
        html.H3("Welcome to the US Aviation Dashboard"),
        html.Hr(),
        html.P(
            "Explore distribution of data by Airports and Airlines", className="lead"
        ),
         html.P(
            "Select Airlines", className="lead"
        ),
          dcc.Dropdown(Airlines,'Delta Airlines',id='Airlines_value',
                multi=False), 
        html.Br(),
         html.P(
            "Select Airport", className="lead"
        ),
         dcc.Dropdown(Airport,'Boston Logan',id='Airport_value',
                multi=False),
        html.Br(),

     ], style = SIDEBAR_STYLE)
                
content = html.Div(style= CONTENT_STYLE, children=[
    html.H1(
        children='U.S. Commercial Aviation Industry Metrics',
        style={
            'textAlign': 'center'   
        }
    ),

    html.Div(children='A web application framework.', style={
        'textAlign': 'center'
    }),
  
    html.Br(),
    
    
     html.Div(children="We are aiming to extract actionable insights from this dataset and be able to get a better understanding of the landscape of the US airplanes and airport industry. We will create a simple, beautiful, and elegant dashboard that will allow people not familiar with the data to quickly dive into it and start extracting patterns.", style={
        'textAlign': 'center', "background-color": "#f8f9fa"
    }),
    
    html.Br(),
    
    html.Div(style = {"background-color": "#f8f9fa"}, children=[
    html.P(style = {'display': 'inline'},children="Showing Airport Statistics and Analyisis For :-"),
    html.H5(id='Airport1',style = {'display': 'inline-block'})]),
    
    html.Div(style = {"background-color": "#f8f9fa"},children=[
    html.P(style = {'display': 'inline-block'},children="For the Following Selected Airlines :- "),
    html.H5(id='Airlines1',style = {'display': 'inline-block'})]),
    
    html.Br(),
    
    html.H3(
            "Bar Graph", className="lead"
        ), html.P("ASM_Domestic: Available Seat-Miles in thousands (000s). Number of domestic flights * Number of seats on each flight"),
    html.P("RPM_Domestic: Revenue Passenger-Miles in thousands (000s). Number of domestic flights"),
    dcc.Graph(id='fig1'),
    
     html.H3(
            "Scatter Metrics", className="lead"
        ), dcc.Graph(id='fig2'),
    
     html.H3(
            "Line Graph", className="lead"
        ), dcc.Graph(id='fig3'),
    
     html.Iframe(
            src="assets/USFlight_Tableau.html",
            style={ "height":"750px", "width": "100%"},
        ),
  
    
    
])
                            
app.layout = html.Div([sidebar, content])

dateparse = lambda dates: pd.datetime.strptime(dates, '%Y-%m-%d')

def get_data(airline, airport, categories=["Passengers"]):
    
    """Takes airline and airport code strings along with one or more data
    category strings as inputs and returns a pandas Series if only one category 
    is requested. Returns pandas DataFrame for calls with more than one category.
    """
    # Date indexes are read in as unicode literals, dataparse will properly
    # reconvert them to DatetimeIndex format
    
    data = pd.read_csv('data_all/{0}-{1}.csv'.format(airline, airport), index_col='Date', 
                       parse_dates=True, date_parser=dateparse)
    
        # Returns DataFrame if more than one category is requested
    if len(categories) > 1:
        columns = ['{}_Domestic'.format(category) for category in categories]
        return data[columns].astype(np.float64)
    
    # Returns Series if only one category is requested
    else:
        return data['{}_Domestic'.format(categories[0])].astype(np.float64)

@app.callback([
    Output('Airport1','children'),
    Output('Airlines1','children'),
    Output('fig1','figure'),
    Output('fig2','figure'),
    Output('fig3','figure')
    ],
    [Input('Airlines_value', 'value'),
    Input('Airport_value','value')])

def update_map(Airlines_value, Airport_value):    
    
    dff1 = df1.loc[df1['Airline Name'] == Airlines_value]
    Airlines_code = dff1['Airline Code'].to_string()
    Airlines_code = Airlines_code.replace(" ", "")
    Airlines_code = Airlines_code[1:]\
    
    dff2 = df2.loc[df2['Airport Name'] == Airport_value]
    Airport_code = dff2['Airport Code'].to_string()
    Airport_code = Airport_code.replace(" ", "")
    Airport_code = Airport_code[1:]\
    
    Airport1 = Airport_value 
    Airlines1 = Airlines_value
    
    data_table = get_data(Airlines_code, Airport_code, categories=['Passengers','ASM', 'RPM'])
    data_table_sc = data_table.divide(1000)
    
    fig1 = px.bar(data_table_sc)
    fig2 = px.scatter(data_table_sc)
    fig3 = px.line(data_table_sc)
    


    return Airport1, Airlines1, fig1, fig2, fig3
          
app.run_server(debug=True, use_reloader=False) 

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
